In [1]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd


2023-06-14 15:29:53.781104: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
print("Tensorflow version: ", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
print(get_available_gpus())

Tensorflow version:  2.4.1
Num GPUs Available:  1
['/device:GPU:0']


2023-06-14 15:30:29.801676: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-06-14 15:30:29.804032: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-06-14 15:30:31.780587: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-14 15:30:31.780825: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:61:00.0 name: NVIDIA GeForce RTX 3080 computeCapability: 8.6
coreClock: 1.71GHz coreCount: 68 deviceMemorySize: 9.77GiB deviceMemoryBandwidth: 707.88GiB/s
2023-06-14 15:30:31.780845: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-06-14 15:30:32.405454: I tensorflow/stream_executor/platform/def

In [3]:
def process_csv_file(file):
    try:
     csv_data = pd.read_csv(file)
    except:
         print("CANT READ CSV: ", file)
         return
    
    if csv_data.isnull().values.any():
            print(file)
            return False
    try:
        csv_data = csv_data.drop("class", axis = 1)
    except KeyError:
        pass
    return (csv_data.to_numpy())

In [13]:
!cd /Tmp/ASL-data/Database-augmented/coffee/; ls;
!ls

2sGQuduhAf41354.csv
2sGQuduhAf41354.csv_projective_geo_12_False.csv
2sGQuduhAf41354.csv_projective_geo_6_True.csv
2sGQuduhAf41354_reflection.csv
2sGQuduhAf41354_reflection.csv_projective_geo_14_False.csv
2sGQuduhAf41354_reflection.csv_projective_geo_8_True.csv
2sGQuduhAf41354_reflection_rotation_15.csv
2sGQuduhAf41354_reflection_rotation_15.csv_projective_geo_7_False.csv
2sGQuduhAf41354_reflection_rotation_15.csv_projective_geo_9_True.csv
2sGQuduhAf41354_reflection_rotation_15_scale_0.2.csv
2sGQuduhAf41354_reflection_rotation_15_scale_0.2.csv_projective_geo_10_False.csv
2sGQuduhAf41354_reflection_rotation_15_scale_0.2.csv_projective_geo_10_True.csv
2sGQuduhAf41354_reflection_rotation_15_scale_0.2_translate_0.5_left.csv
2sGQuduhAf41354_reflection_rotation_15_scale_0.2_translate_0.5_left.csv_projective_geo_10_True.csv
2sGQuduhAf41354_reflection_rotation_15_scale_0.2_translate_0.5_left.csv_projective_geo_8_False.csv
2sGQuduhAf41354_reflection_rotation_15_translate_0.5_left.csv
2sGQuduhAf4

In [30]:
file = ""
process_csv_file(file)

array([[ 0.51505029,  0.337479  , -0.79841799, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.5149141 ,  0.33766055, -0.79397482, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.51484638,  0.33796486, -0.76542687, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.52175295,  0.3408646 , -0.72429359, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.52186459,  0.34013429, -0.74814087, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.52185893,  0.33947977, -0.76047277, ...,  0.        ,
         0.        ,  0.        ]])

In [31]:
def set_padding(arr, max_length):
    arr = np.append(arr, np.zeros((max_length-arr.shape[0],300)), axis=0)
    return np.expand_dims(arr, 0)

In [32]:
def prepare_data(folder_path, labels, train=True):
    total = 0
    max_length = 211
    
    # find max length
    # for folder in os.listdir(folder_path):
    #     print(folder)
    #     if folder != ".DS_Store":
    #         total += len(os.listdir(os.path.join(folder_path, folder)))
    #         for file in os.listdir(os.path.join(folder_path, folder)):
    #             if file != ".DS_Store":
    #                 data = process_csv_file(os.path.join(folder_path, folder, file))
    #                 if data is not(False):
    #                     if data.shape[0] > max_length:
    #                         max_length = data.shape[0]
    # print(max_length)
    # Make the arrays
    X = np.empty((0, max_length, 300))
    Y = np.empty((0,), int)
    print("----------")
    for folder in os.listdir(folder_path):
        print(folder)
        if folder != ".DS_Store":
            i=0
            for file in os.listdir(os.path.join(folder_path, folder)):
                if file != ".DS_Store":
                    if (train==True and i%5 != 0) or (train==False and i%5==0):
                        data = process_csv_file(os.path.join(folder_path, folder, file))
                        if data is not(False):
                            data = set_padding(data, max_length)
                            X = np.vstack((X, data))
                            Y = np.append(Y, labels[folder])
                    i+=1
    print(total, max_length)
    return X, Y

In [33]:
folder_path_train = "/Tmp/ASL-data/Database-augmented/"
labels= {"coffee": 0, 'dog': 1, 'milk': 2, 'door': 3}
X_train, Y_train = prepare_data(folder_path_train, labels)

----------
door
/Tmp/ASL-data/Database-augmented/door/WGfiiDgrq1I3897_rotation_15_translate_0.5_left.csv_projective_geo_11_False.csv
/Tmp/ASL-data/Database-augmented/door/WGfiiDgrq1I3897_reflection_rotation_15.csv_projective_geo_8_False.csv
/Tmp/ASL-data/Database-augmented/door/WGfiiDgrq1I3897_rotation_15_translation_0.5.csv_projective_geo_8_True.csv
/Tmp/ASL-data/Database-augmented/door/WGfiiDgrq1I3897_reflection_scale_0.2_translate_0.5_left.csv_projective_geo_10_False.csv
/Tmp/ASL-data/Database-augmented/door/WGfiiDgrq1I3897_reflection_rotation_15.csv_projective_geo_8_True.csv
/Tmp/ASL-data/Database-augmented/door/WGfiiDgrq1I3897_rotation_15_translation_0.5_translate_0.5_left.csv_projective_geo_12_False.csv
/Tmp/ASL-data/Database-augmented/door/WGfiiDgrq1I3897_rotation_15.csv_projective_geo_13_True.csv
/Tmp/ASL-data/Database-augmented/door/WGfiiDgrq1I3897_reflection_rotation_15_translation_0.5_scale_0.2.csv_projective_geo_12_True.csv
/Tmp/ASL-data/Database-augmented/door/WGfiiDgrq1I3

array([], shape=(0, 211, 300), dtype=float64)

In [34]:
folder_path_test = "/Tmp/ASL-data/Database-augmented/"
labels = {"coffee": 0, 'dog': 1, 'milk': 2, 'door': 3}
X_test, Y_test = prepare_data(folder_path_test, labels, train=True)

----------
door
/Tmp/ASL-data/Database-augmented/door/WGfiiDgrq1I3897_rotation_15_translate_0.5_left.csv_projective_geo_11_False.csv
/Tmp/ASL-data/Database-augmented/door/WGfiiDgrq1I3897_reflection_rotation_15.csv_projective_geo_8_False.csv
/Tmp/ASL-data/Database-augmented/door/WGfiiDgrq1I3897_rotation_15_translation_0.5.csv_projective_geo_8_True.csv
/Tmp/ASL-data/Database-augmented/door/WGfiiDgrq1I3897_reflection_scale_0.2_translate_0.5_left.csv_projective_geo_10_False.csv
/Tmp/ASL-data/Database-augmented/door/WGfiiDgrq1I3897_reflection_rotation_15.csv_projective_geo_8_True.csv
/Tmp/ASL-data/Database-augmented/door/WGfiiDgrq1I3897_rotation_15_translation_0.5_translate_0.5_left.csv_projective_geo_12_False.csv
/Tmp/ASL-data/Database-augmented/door/WGfiiDgrq1I3897_rotation_15.csv_projective_geo_13_True.csv
/Tmp/ASL-data/Database-augmented/door/WGfiiDgrq1I3897_reflection_rotation_15_translation_0.5_scale_0.2.csv_projective_geo_12_True.csv
/Tmp/ASL-data/Database-augmented/door/WGfiiDgrq1I3

In [13]:
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    [
    [711, 632, 71],
    [73, 8, 3215, 55, 927],
    [83, 91, 1, 645, 1253, 927],
], padding="post"
)
# print(X[0][1])
print(((padded_inputs)))

[[ 711  632   71    0    0    0]
 [  73    8 3215   55  927    0]
 [  83   91    1  645 1253  927]]


In [24]:
# Y = np.expand_dims(Y, axis=0)
# Y= Y.T
Y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [35]:
# shuffle data
from sklearn.utils import shuffle

X_train,Y_train = shuffle(X_train, Y_train, random_state=0)
X_val,Y_val = shuffle(X_test, Y_test, random_state=0)
X_train, Y_train

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
X_train.shape, Y_train.shape

((11060, 211, 300), (11060,))

In [ ]:
x_shape = X_train.shape

In [87]:
from tensorflow.keras import layers
masking_input = []
for i in range(x_shape[0]):
    masking_input.append(X_train[i].T[0])

masking_input = np.array(masking_input)

masking_input.shape

(11060, 211)

In [20]:
embedding = layers.Embedding(input_dim=x_shape[1], output_dim=x_shape[1], mask_zero=True)
masked_output = embedding(masking_input)

print(masked_output._keras_mask)
masked_output
# masking_layer = layers.Masking()
# # Simulate the embedding lookup by expanding the 2D input to 3D,
# # with embedding dimension of 10.
# unmasked_embedding = tf.cast(
#     tf.tile(tf.expand_dims(padded_inputs, axis=-1), [1, 1, 10]), tf.float32
# )

# masked_embedding = masking_layer(unmasked_embedding)
# print(masked_embedding._keras_mask)

2023-04-21 00:13:36.221629: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-21 00:13:36.222041: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

tf.Tensor(
[[ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 ...
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]], shape=(457, 211), dtype=bool)


<tf.Tensor: shape=(457, 211, 211), dtype=float32, numpy=
array([[[ 0.04668217,  0.04604015,  0.01552759, ...,  0.0464903 ,
         -0.01109668, -0.01088526],
        [ 0.04668217,  0.04604015,  0.01552759, ...,  0.0464903 ,
         -0.01109668, -0.01088526],
        [ 0.04668217,  0.04604015,  0.01552759, ...,  0.0464903 ,
         -0.01109668, -0.01088526],
        ...,
        [ 0.04668217,  0.04604015,  0.01552759, ...,  0.0464903 ,
         -0.01109668, -0.01088526],
        [ 0.04668217,  0.04604015,  0.01552759, ...,  0.0464903 ,
         -0.01109668, -0.01088526],
        [ 0.04668217,  0.04604015,  0.01552759, ...,  0.0464903 ,
         -0.01109668, -0.01088526]],

       [[ 0.04668217,  0.04604015,  0.01552759, ...,  0.0464903 ,
         -0.01109668, -0.01088526],
        [ 0.04668217,  0.04604015,  0.01552759, ...,  0.0464903 ,
         -0.01109668, -0.01088526],
        [ 0.04668217,  0.04604015,  0.01552759, ...,  0.0464903 ,
         -0.01109668, -0.01088526],
        ..

In [ ]:
#LSTM model 
from tensorflow.keras import layers
x_shape = (457, 211, 300)
inputs = layers.Input(shape=(x_shape[1], x_shape[2]))
x = layers.Masking(mask_value=0, input_shape=(x_shape[1], x_shape[2]))(inputs)
#return_sequences makes so that we can stack lstm layers together. Since the input shape of lstm is different
x = layers.LSTM(64, return_sequences = True)(x)
x = layers.LSTM(32, return_sequences = True)(x)
x = layers.Dense(64)(x)
# x = layers.MaxPool2D()(x)
outputs = layers.Dense(4, activation="softmax")(x)

model_lstm = tf.keras.Model(inputs, outputs)
model_lstm.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 211, 300)]        0         
                                                                 
 masking_1 (Masking)         (None, 211, 300)          0         
                                                                 
 lstm_2 (LSTM)               (None, 211, 64)           93440     
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
 dense_2 (Dense)             (None, 64)                2112      
                                                                 
 dense_3 (Dense)             (None, 4)                 260       
                                                                 
Total params: 108,228
Trainable params: 108,228
Non-trainab

In [ ]:
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.fit(X_train, Y_train, epochs=50, validation_data=(X_val, Y_val))

Epoch 1/50


2023-05-16 07:19:40.722082: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-05-16 07:19:42.567144: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-05-16 07:19:48.025183: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-05-16 07:19:53.578057: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-05-16 07:21:11.596147: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 10/346 [..............................] - ETA: 3:22 - loss: 1.3942 - accuracy: 0.2594

In [47]:
def get_mask_from_lengths(data):  #2d list
    ids = tf.zeros(shape=(x_shape[1], x_shape[2]))
    all_zeros = ids < data
    mask_list = []
    for i in range()
    return mask

get_mask_from_lengths(X_train)[0]

<tf.Tensor: shape=(211, 300), dtype=bool, numpy=
array([[ True,  True, False, ...,  True, False, False],
       [ True,  True, False, ...,  True, False, False],
       [ True,  True, False, ...,  True, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])>

In [48]:
inputs = layers.Input(shape=(x_shape[1], x_shape[2]))
x = layers.GRU(64)(inputs, mask=masked_output._keras_mask)
# x = layers.LSTM(64)(x)
outputs = layers.Dense(4, activation="softmax")(x)
model_lstm2= tf.keras.Model(inputs, outputs)
model_lstm2.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 211, 300)]        0         
                                                                 
 gru (GRU)                   (114, 64)                 70272     
                                                                 
 dense_3 (Dense)             (114, 4)                  260       
                                                                 
Total params: 70,532
Trainable params: 70,532
Non-trainable params: 0
_________________________________________________________________


In [49]:
model_lstm2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm2.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5


2023-03-25 16:08:52.820913: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-25 16:08:53.187428: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


CancelledError: Graph execution error:

RecvAsync is cancelled.
	 [[{{node model_3/dense_3/Softmax/_20}}]] [Op:__inference_train_function_36100]

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(4, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])




Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-03-20 13:48:08.831618: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-20 13:48:08.831726: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


NameError: name 'X_train' is not defined

In [41]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

model.compile(loss=SparseCategoricalCrossentropy(), optimizer=Adam(), metrics=["accuracy"])
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_33 (LSTM)              (None, 211, 64)           93440     
                                                                 
 lstm_34 (LSTM)              (None, 32)                12416     
                                                                 
 dense_7 (Dense)             (None, 4)                 132       
                                                                 
Total params: 105,988
Trainable params: 105,988
Non-trainable params: 0
_________________________________________________________________


In [42]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/10


2023-03-18 02:36:51.372559: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-18 02:36:51.660659: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-18 02:36:51.934093: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-18 02:36:52.419899: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-18 02:36:53.015099: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - ETA: 0s - loss: 1.3853 - accuracy: 0.2353

2023-03-18 02:36:55.555320: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-18 02:36:55.667921: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-18 02:36:55.926320: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 6s 863ms/step - loss: 1.3853 - accuracy: 0.2353 - val_loss: 1.3813 - val_accuracy: 0.3333
Epoch 2/10
4/4 [==============================] - 0s 108ms/step - loss: 1.3759 - accuracy: 0.3333 - val_loss: 1.3774 - val_accuracy: 0.3333
Epoch 3/10
4/4 [==============================] - 0s 99ms/step - loss: 1.3661 - accuracy: 0.3431 - val_loss: 1.3739 - val_accuracy: 0.3333
Epoch 4/10
4/4 [==============================] - 0s 101ms/step - loss: 1.3470 - accuracy: 0.3431 - val_loss: 1.3667 - val_accuracy: 0.3333
Epoch 5/10
4/4 [==============================] - 0s 101ms/step - loss: 1.3325 - accuracy: 0.3431 - val_loss: 1.3640 - val_accuracy: 0.3333
Epoch 6/10
4/4 [==============================] - 0s 101ms/step - loss: 1.3215 - accuracy: 0.3431 - val_loss: 1.4380 - val_accuracy: 0.3333
Epoch 7/10
4/4 [==============================] - 0s 102ms/step - loss: 1.2628 - accuracy: 0.4118 - val_loss: 2.1948 - val_accuracy: 0.1667
Epoch 8/10
4/4 [================

In [38]:
x = np.array([[4], [2]])

In [41]:
x.T.shape

(1, 2)

In [42]:
x.T

array([[4, 2]])